In [62]:
import pandas as pd
import numpy as np

data = pd.read_csv('latency.log').to_numpy().squeeze()
min_val = np.min(data)
max_val = np.max(data)
median_val = np.mean(data)
mean_val = np.mean(data)
std_val = np.std(data)
p_vals = np.percentile(data, [25, 50, 75, 90, 95, 99])

vals = [min_val, max_val, median_val, mean_val, std_val]
vals.extend(p_vals)
vals = pd.DataFrame(vals).round(2).T
vals.columns = ['min','max','median','mean','std','25th','50th','75th','90th','95th','99th']


   min   max  median  mean  std  25th  50th  75th  90th  95th  99th
0  0.2  3.79    1.19  1.19  0.5  0.83  1.12  1.44  1.85  2.12  2.88


In [ ]:
#!/usr/bin/env python3
import pandas as pd
import json
import subprocess
from influxdb import InfluxDBClient
import numpy as np

def find_influxdb(influxdb):
    cnr_pid = subprocess.check_output("sudo docker inspect --format '{{ .State.Pid }}' %s" % influxdb,
                                      shell=True).decode().strip()
    cnr_ip = \
    subprocess.check_output("sudo nsenter -t %s -n ip addr | grep inet | awk '{print $2}' | tail -n 1" % cnr_pid,
                            shell=True).decode().strip().split('/')[0]
    return cnr_ip

if __name__ == '__main__':
    client = InfluxDBClient(host=find_influxdb('fecbench-server-docker_influxdb_1'), port=8086,
                            username='indices_user',
                            password='indices_manager', database='collectd_db')

    kafka_metrics_query_str = 'SELECT * FROM "docker_kafka_metrics" WHERE time > now()-120s AND time < now() GROUP BY "instance"'
    kafka_metrics = client.query(query=kafka_metrics_query_str, database='collectd_db')
    kafka_raw_metrics = kafka_metrics.raw['series']
    for i in range(len(kafka_raw_metrics)):
        instance = kafka_raw_metrics[i]['tags']['instance']
        if 'g-1-s-0' in instance:
            kafka_metrics = list(kafka_metrics.get_points(tags={"instance": instance}))
            kafka_metrics = pd.DataFrame(kafka_metrics)
            break
            print(kafka_metrics)
